In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import requests
import json
from census import Census
import gmaps

# Import API key
from config import g_key


In [ ]:
# Import the fast food data file 
fastfood_data = pd.read_csv('resources/clean_fast_food.csv')
fastfood_data


In [ ]:
# Import the combined fast food - obesity - population data file 
combined_fastfood= pd.read_csv('resources/clean_all_combined.csv')
combined_fastfood


In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
obesity_data= pd.read_csv('resources/obesity_data_state_2020.csv')


In [ ]:
# Build a dataframe that has city, lat and long. Remove the dups. 
locations = fastfood_data[["City", "Latitude", "Longitude"]].drop_duplicates()


In [ ]:
#Group the fast food data by city and count the number of fast food places in each city. 
fast_food_totals = fastfood_data.groupby("City")[['Index']].count()
fast_food_totals

In [ ]:
#rename the index column to the fast food count 
fast_food_totals.rename(columns = {'Index' : 'Fast Food Count'}, inplace = True)


In [ ]:
# create a data frame that has city, fast food count, lat and long - merging the grouped df with the locations
fast_food_by_city= pd.merge(fast_food_totals, locations, how = 'right', on = 'City')
fast_food_by_city.head(30)

In [ ]:
#prep the parameters for teh heat map 
locations = fast_food_by_city[["Latitude", "Longitude"]]
ff_count = fast_food_by_city["Fast Food Count"].astype(float)

In [ ]:

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=ff_count, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
#Julia work - fast food quantity by obesity by state

#group combined df by date
fast_food_restaurant = combined_fastfood.groupby("State")

#find count of fast food chain by state
fast_food_count = fast_food_restaurant["Name"].count()
fast_food_count

#remove state duplicates from combined data
clean_combined_obesity = combined_fastfood.drop_duplicates('State')

#group cleaned dats by state
clean_obesity_group = clean_combined_obesity.groupby("State")

#find obesity rate per state
obesity_rate = clean_obesity_group['Prevalence'].mean()
obesity_rate

plt.scatter(obesity_rate, fast_food_count, marker ="o", facecolors="blue", s=100, alpha=0.75)
plt.xlabel("Obesity Rate")
plt.ylabel("Number of Fast Food Restaurants")
plt.title("Fast Food and Obesity Rates by State")

In [ ]:
#empty list for state
states_count = [combined_fastfood["State"].nunique()]

Obesity_by_state = obesity_rate.plot(kind="bar", title= "Obesity Rate by State", alpha=0.85, rot=78, 
                                     width =0.5, figsize = (15, 10))
Obesity_by_state.set_xlabel("State")
Obesity_by_state.set_ylabel("Obesity Rate")


In [ ]:
#Julia work - fast food quantity by obesity by state

In [ ]:
#Nicole Work - fast food per capita
fast_food = "resources/clean_fast_food.csv"
clean_fast_food = pd.read_csv(fast_food, encoding="utf-8")
clean_fast_food

In [ ]:
url = 'http://api.census.gov/data/2020/dec/pl?get=NAME,H1_001N&for=state:*'

from config import api_key
c = Census(api_key, year=2021)

census_data = requests.get(url).json()

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
census_pd.columns = ["State", "Total Population", "State Number"]
census_pd

In [ ]:
census_pd = census_pd.drop(census_pd.index[0])
census_pd

In [ ]:
resturant_count=clean_fast_food.groupby(["State", "Name"]).count()["Index"]
resturant_count

In [ ]:
state_count = clean_fast_food.groupby("State").count()["Index"]
state_count

In [ ]:
plot_pandas =  state_count.plot.bar (figsize = (15,10), color = 'b', fontsize = 14)
state_count
plt.xlabel('State', fontsize = 14)
plt.ylabel('Number of Fast Food Resturants', fontsize = 14)
plt.title('Number of Fast Food Resturants per State', fontsize =20)
plt.show

In [ ]:
fast_food_census= pd.merge(state_count, census_pd, how = 'inner', on = 'State')
fast_food_census.head()

In [ ]:
fast_food_census.columns = ['State',
'Number of Resturants',
'Total Population',
'Number of Resturants/Tot Population']
                            
fast_food_census.head()

In [ ]:
fast_food_census.dtypes
fast_food_census["Total Population"] =pd.to_numeric(fast_food_census["Total Population"])
fast_food_census["Total Population"].dtype

In [ ]:
fast_food_census ["Number of Resturants/Tot Population"] = fast_food_census ["Number of Resturants"]/ fast_food_census ["Total Population"]
fast_food_census

In [ ]:
plot_ff_capita =  fast_food_census.plot.scatter ('State','Number of Resturants/Tot Population',figsize = (15,10), color = 'b', fontsize = 14)
plt.xlabel('State', fontsize = 14)
plt.xticks(rotation=90)
plt.ylabel('Number of Resturants/Tot Population', fontsize = 14)
plt.title('Fast Food Resturants per Capita', fontsize =20)
plt.show()